# Deep Learning A Z Regressão 1 valor - Automóveis Cruzada

In [ ]:
!pip install tensorflow==2.16.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [ ]:
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [ ]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.0.3', '2.16.1', '1.5.0', '0.13.0')

In [ ]:
import time
from scikeras.wrappers import KerasRegressor
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

In [ ]:
inicio = time.time()

In [ ]:
inicio

1719411287.5327137

In [ ]:
base = pd.read_csv('autos.csv', encoding='ISO-8859-1')

In [ ]:
base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis=1)
base = base.drop('nrOfPictures', axis=1)
base = base.drop('postalCode', axis=1)
base = base.drop('lastSeen', axis=1)
base = base.drop('name', axis=1)
base = base.drop('seller', axis=1)
base = base.drop('offerType', axis=1)

In [ ]:
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [ ]:
valores = {'vehicleType': 'limousine',
           'gearbox': 'manuell',
           'model': 'golf',
           'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value=valores)

In [ ]:
X = base.iloc[:, 1:12].values
y = base.iloc[:, 0].values

In [ ]:
X, y

(array([['test', 'limousine', 1993, ..., 'benzin', 'volkswagen', 'nein'],
        ['test', 'coupe', 2011, ..., 'diesel', 'audi', 'ja'],
        ['test', 'suv', 2004, ..., 'diesel', 'jeep', 'nein'],
        ...,
        ['test', 'bus', 1996, ..., 'diesel', 'volkswagen', 'nein'],
        ['test', 'kombi', 2002, ..., 'diesel', 'volkswagen', 'nein'],
        ['control', 'limousine', 2013, ..., 'benzin', 'bmw', 'nein']],
       dtype=object),
 array([  480, 18300,  9800, ...,  9200,  3400, 28990]))

In [ ]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder='passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [ ]:
X.shape

(359291, 316)

In [ ]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
        tf.keras.layers.InputLayer(shape=(316,)),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='linear')])
    regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    return regressor

In [ ]:
regressor = KerasRegressor(model = criar_rede, epochs = 100, batch_size = 300)

In [ ]:
resultados = cross_val_score(estimator = regressor, X = X, y = y,
                             cv = 5, scoring = 'neg_mean_absolute_error')

Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 4349.8164 - mean_absolute_error: 4349.8164
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 3527.9626 - mean_absolute_error: 3527.9626
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3320.6409 - mean_absolute_error: 3320.6409
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 3046.9976 - mean_absolute_error: 3046.9976
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 2891.4280 - mean_absolute_error: 2891.4280
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 2805.6428 - mean_absolute_error: 2805.6428
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 2752.0898 - mean_absolute_error: 2752.0898
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 2719.9910 - mean_absolute_error: 2719.9910
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 2701.6902 - mean_absolute_error: 2701.6902
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 

In [ ]:
fim = time.time()

In [ ]:
fim

1719416624.8128402

In [ ]:
(fim - inicio) / 60 / 60

1.482577812936571

In [ ]:
abs(resultados)

array([2234.82849182, 2526.7743402 , 2232.3855782 , 2406.84199151,
       2228.20925244])

In [ ]:
abs(resultados.mean())

2325.807930835687

In [ ]:
resultados.std()

121.23078239768839